<a href="https://colab.research.google.com/github/newmantic/Longstaff_Schwartz/blob/main/Longstaff_Schwartz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy.polynomial.laguerre import lagval

def longstaff_schwartz_american_option(S0, K, r, sigma, T, M, N, option_type='call'):
    """
    Longstaff-Schwartz method for pricing American options.

    Parameters:
    - S0: Initial stock price
    - K: Strike price
    - r: Risk-free interest rate (annual)
    - sigma: Volatility of the underlying asset (annual)
    - T: Time to maturity (in years)
    - M: Number of simulations
    - N: Number of time steps
    - option_type: 'call' for call option, 'put' for put option

    Returns:
    - option_price: The estimated price of the American option
    """
    dt = T / N
    discount_factor = np.exp(-r * dt)

    # Simulate asset price paths
    S = np.zeros((M, N + 1))
    S[:, 0] = S0
    for t in range(1, N + 1):
        Z = np.random.normal(0, 1, M)
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

    # Initialize option values at maturity
    if option_type == 'call':
        V = np.maximum(S[:, -1] - K, 0)
    elif option_type == 'put':
        V = np.maximum(K - S[:, -1], 0)
    else:
        raise ValueError("Invalid option type. Choose 'call' or 'put'.")

    # Step backwards in time to estimate the option price
    for t in range(N-1, 0, -1):
        in_the_money = np.where(S[:, t] > K if option_type == 'call' else S[:, t] < K)[0]
        X = S[in_the_money, t]
        Y = discount_factor * V[in_the_money]

        if len(X) > 0:
            # Regression to estimate continuation value
            A = np.vander(X, 4)
            C = np.linalg.lstsq(A, Y, rcond=None)[0]
            continuation_value = lagval(X, C)

            exercise_value = np.maximum(S[in_the_money, t] - K, 0) if option_type == 'call' else np.maximum(K - S[in_the_money, t], 0)
            V[in_the_money] = np.where(exercise_value > continuation_value, exercise_value, discount_factor * V[in_the_money])

    # Option price is the discounted expected payoff
    option_price = discount_factor * np.mean(V)

    return option_price

In [2]:
# Parameters
S0 = 100         # Initial stock price
K = 100          # Strike price
r = 0.05         # Risk-free interest rate (5% per annum)
sigma = 0.2      # Volatility (20% per annum)
T = 1            # Time to maturity (1 year)
M = 10000        # Number of simulations
N = 50           # Number of time steps

# Price the American call option
call_price = longstaff_schwartz_american_option(S0, K, r, sigma, T, M, N, option_type='call')
print(f"American Call Option Price: {call_price:.2f}")

# Price the American put option
put_price = longstaff_schwartz_american_option(S0, K, r, sigma, T, M, N, option_type='put')
print(f"American Put Option Price: {put_price:.2f}")

American Call Option Price: 2.09
American Put Option Price: 4.37
